# 🤖 Nemotron-VLA: Vision-Language-Action Model with NVIDIA Foundation Models<div align="center">**A fully-functional VLA powered by NVIDIA Nemotron & RADIO***Vision: NVIDIA RADIO · Language: NVIDIA Nemotron Nano 9B v2 · Action: Diffusion Policy*[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](#)</div>---## Overview**Nemotron-VLA** is a Vision-Language-Action model that uses NVIDIA's foundation models as its backbone:| Modality | Model | Role | Parameters | Trainable? ||----------|-------|------|------------|------------|| 🖼️ Vision | **NVIDIA RADIO** | Image feature extractor | ~0.4-0.7B | ❄️ Frozen || 📝 Language | **NVIDIA Nemotron Nano 9B v2** | Text instruction encoder | ~9B | ❄️ Frozen || 🦾 Action | **Diffusion Policy Head** | Action generation | ~0.5M | 🔥 Trained || 🔀 Fusion | **Cross-Attention Module** | Multi-modal fusion | ~0.3M | 🔥 Trained |### Architecture```  NVIDIA RADIO (ViT)    Nemotron Nano 9B v2    Robot State       │                       │                    │   [frozen]                [frozen]             [trainable]       │                       │                    │  Vision Proj ──────── Text Proj ──────── State Encoder       │                       │                    │       └───────────┬───────────┘────────────────────┘                   │          Cross-Attention Fusion                   │         Diffusion Policy Head                   │              Robot Actions```### Training StrategyTo fit everything on an **A100 40GB** Colab GPU:1. **Collect** expert demonstrations from MetaWorld2. **Precompute** RADIO vision features (load → extract → unload)3. **Precompute** Nemotron text embeddings (load → extract → unload)4. **Train** only the lightweight fusion + diffusion modules> 💡 **Key insight:** By precomputing embeddings from the frozen NVIDIA models, we only need ~1M trainable parameters during the training loop, making it extremely memory efficient!

---## 1. 🔧 Setup & InstallationFirst, let's install all required dependencies. This cell handles MetaWorld, MuJoCo, NVIDIA models, and rendering on Colab.

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1.1 System dependencies for MuJoCo headless rendering
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Install BOTH EGL and OSMesa so auto-detection works everywhere
!apt-get update -qq
!apt-get install -y -qq \
    libegl1-mesa-dev \
    libosmesa6-dev \
    libgl1-mesa-glx \
    libglfw3 \
    libglew-dev \
    patchelf \
    > /dev/null 2>&1

print("✅ System rendering libraries installed (EGL + OSMesa)")


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1.2 Python packages
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
!pip install -q \
    torch torchvision \
    transformers accelerate \
    mujoco \
    gymnasium metaworld \
    imageio[ffmpeg] imageio-ffmpeg \
    matplotlib tqdm

# For Nemotron's hybrid Mamba-2 architecture
!pip install -q causal-conv1d mamba-ssm

print("\n✅ All Python packages installed")


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1.3 Verify GPU & rendering backend
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available:  {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    mem = torch.cuda.get_device_properties(0).total_mem / 1e9
    print(f"GPU:             {gpu}")
    print(f"GPU Memory:      {mem:.1f} GB")

# ── Rendering backend diagnostic ──
import ctypes, os

egl_ok, osmesa_ok = False, False
try:
    ctypes.cdll.LoadLibrary("libEGL.so")
    egl_ok = True
except OSError:
    pass
try:
    ctypes.cdll.LoadLibrary("libOSMesa.so")
    osmesa_ok = True
except OSError:
    pass

print(f"\nRendering backends:")
print(f"  EGL:    {'✅ available' if egl_ok else '❌ not found'}")
print(f"  OSMesa: {'✅ available' if osmesa_ok else '❌ not found'}")

# Auto-select: EGL preferred (hardware accel on NVIDIA GPUs)
if egl_ok:
    os.environ["MUJOCO_GL"] = "egl"
    print(f"\n→ Using EGL (hardware-accelerated, native on NVIDIA GPUs)")
elif osmesa_ok:
    os.environ["MUJOCO_GL"] = "osmesa"
    os.environ["PYOPENGL_PLATFORM"] = "osmesa"
    print(f"\n→ Using OSMesa (software rendering fallback)")
else:
    raise RuntimeError("No rendering backend found! Install libegl1-mesa-dev or libosmesa6-dev")

# Verify MuJoCo imports cleanly with the chosen backend
import mujoco
print(f"  MuJoCo {mujoco.__version__} loaded with MUJOCO_GL={os.environ['MUJOCO_GL']}")
print("\n🚀 Ready to go!")


---## 2. 📁 Write Helper ModulesWe organize the code into three helper files:- **`env.py`** — MetaWorld environment wrapper & data collection- **`models.py`** — RADIO encoder, Nemotron encoder, fusion, diffusion head, full VLA- **`utils.py`** — Dataset, training loop, evaluation, visualization

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Create project directories
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import os
for d in ["data", "checkpoints", "videos", "embeddings"]:
    os.makedirs(d, exist_ok=True)
print("📂 Project directories created")


### 2.1 `env.py` — Environment

In [ ]:
%%writefile env.py
"""
Nemotron-VLA: Environment Wrapper
=================================
MetaWorld MT1 environment wrapper for robot manipulation tasks.
Compatible with Meta-World's gymnasium interface.
"""

import os

def _setup_rendering():
    """
    Auto-detect the best MuJoCo rendering backend.
    - EGL:    hardware-accelerated, works on NVIDIA GPUs (Colab, clusters)
    - OSMesa: software fallback, needs libOSMesa.so
    """
    # If user already set it, respect their choice
    if "MUJOCO_GL" in os.environ:
        return

    # Try EGL first (native on NVIDIA GPUs, no PyOpenGL needed)
    try:
        import ctypes
        ctypes.cdll.LoadLibrary("libEGL.so")
        os.environ["MUJOCO_GL"] = "egl"
        return
    except OSError:
        pass

    # Try OSMesa as fallback
    try:
        import ctypes
        ctypes.cdll.LoadLibrary("libOSMesa.so")
        os.environ["MUJOCO_GL"] = "osmesa"
        os.environ["PYOPENGL_PLATFORM"] = "osmesa"
        return
    except OSError:
        pass

    # Last resort: let MuJoCo figure it out (may need a display)
    os.environ["MUJOCO_GL"] = "egl"

_setup_rendering()

import numpy as np


class MetaWorldMT1Wrapper:
    """
    Wraps a MetaWorld MT1 environment into a simple interface:
      - reset() -> (image, state, info)
      - step(action) -> (image, state, reward, done, info)
    """

    def __init__(
        self,
        env_name="push-v3",
        seed=42,
        render_mode="rgb_array",
        camera_name="corner2",
    ):
        import gymnasium as gym
        import metaworld  # noqa: F401 — registers Meta-World envs

        self.env = gym.make(
            "Meta-World/MT1",
            env_name=env_name,
            seed=seed,
            render_mode=render_mode,
            camera_name=camera_name,
        )
        self.render_mode = render_mode
        self.env_name = env_name

        obs, _ = self.env.reset()
        self.state_dim = self._extract_state(obs).shape[0]
        self.action_dim = self.env.action_space.shape[0]
        self.obs_shape = self._get_image().shape

    def _extract_state(self, obs):
        """Extract flat state vector from observation."""
        if isinstance(obs, dict):
            if "observation" in obs:
                state = obs["observation"]
            elif "robot_state" in obs or "object_state" in obs:
                parts = []
                if "robot_state" in obs:
                    parts.append(obs["robot_state"])
                if "object_state" in obs:
                    parts.append(obs["object_state"])
                state = np.concatenate(parts, axis=-1)
            else:
                raise KeyError(f"Unknown obs keys: {list(obs.keys())}")
        else:
            state = obs
        return np.asarray(state, dtype=np.float32)

    def _get_image(self):
        """Render and return RGB image."""
        img = self.env.render()
        return img.astype(np.uint8)

    def reset(self, seed=None):
        obs, info = self.env.reset(seed=seed)
        state = self._extract_state(obs)
        image = self._get_image()
        return image, state, info

    def step(self, action):
        obs, reward, truncate, terminate, info = self.env.step(action)
        done = truncate or terminate
        state = self._extract_state(obs)
        image = self._get_image()
        return image, state, reward, done, info

    def close(self):
        self.env.close()


def collect_demonstrations(
    env_name="push-v3",
    camera_name="corner2",
    seed=42,
    num_episodes=100,
    max_steps=150,
    instruction="push the object to the goal",
):
    """
    Collect expert demonstrations using MetaWorld's built-in policies.

    Returns:
        dict with keys: images, states, actions, instruction
    """
    from metaworld.policies import ENV_POLICY_MAP
    import gymnasium as gym
    import metaworld  # noqa: F401

    env = gym.make(
        "Meta-World/MT1",
        env_name=env_name,
        seed=seed,
        render_mode="rgb_array",
        camera_name=camera_name,
    )

    obs, info = env.reset(seed=seed)
    policy = ENV_POLICY_MAP[env_name]()

    all_images, all_states, all_actions = [], [], []

    for ep in range(num_episodes):
        obs, info = env.reset()
        done = False
        steps = 0

        while not done and steps < max_steps:
            action = policy.get_action(obs)
            img = env.render()

            all_images.append(img.astype(np.uint8).copy())
            all_states.append(np.asarray(obs, dtype=np.float32).ravel().copy())
            all_actions.append(np.asarray(action, dtype=np.float32).copy())

            obs, reward, truncate, terminate, info = env.step(action)
            done = bool(truncate or terminate) or (int(info.get("success", 0)) == 1)
            steps += 1

        success = int(info.get("success", 0))
        print(f"  Episode {ep+1}/{num_episodes}: {steps} steps, success={success}")

    env.close()

    data = {
        "images": np.stack(all_images, axis=0),      # (N, H, W, 3)
        "states": np.stack(all_states, axis=0),       # (N, state_dim)
        "actions": np.stack(all_actions, axis=0),     # (N, action_dim)
        "instruction": instruction,
    }

    print(f"\n✅ Collected {data['images'].shape[0]} transitions")
    print(f"   images:  {data['images'].shape}")
    print(f"   states:  {data['states'].shape}")
    print(f"   actions: {data['actions'].shape}")

    return data


### 2.2 `models.py` — Model Components

In [ ]:
%%writefile models.py
"""
Nemotron-VLA: Model Components
===============================
Vision-Language-Action model powered by NVIDIA foundation models:
  - Vision:   NVIDIA RADIO (frozen feature extractor)
  - Language:  NVIDIA Nemotron Nano (frozen text encoder)
  - Action:   Diffusion Policy head (trainable)
  - Fusion:   Cross-attention module (trainable)
"""

import math
from dataclasses import dataclass
from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F


# ╔══════════════════════════════════════════════════════════════════╗
# ║  1. VISION ENCODER — NVIDIA RADIO (frozen)                      ║
# ╚══════════════════════════════════════════════════════════════════╝

def load_radio_model(device="cuda", dtype=torch.float16):
    """
    Load NVIDIA RADIO vision foundation model from HuggingFace.

    RADIO (AM-RADIO) distills knowledge from multiple teacher models
    (CLIP, DINOv2, SAM) into a single efficient ViT backbone.
    It produces both a summary embedding and spatial features.

    Returns:
        model: RADIO model (frozen, eval mode)
        radio_dim: int, dimension of the summary embedding
    """
    from transformers import AutoModel

    print("📸 Loading NVIDIA RADIO vision encoder...")
    model = AutoModel.from_pretrained(
        "nvidia/RADIO",
        trust_remote_code=True,
        torch_dtype=dtype,
    ).to(device).eval()

    # Freeze all parameters
    for p in model.parameters():
        p.requires_grad = False

    # Detect output dimension with a dummy forward pass
    with torch.no_grad():
        dummy = torch.randn(1, 3, 224, 224, device=device, dtype=dtype)
        summary, spatial = model(dummy)
        radio_dim = summary.shape[-1]

    print(f"   ✅ RADIO loaded — summary dim: {radio_dim}")
    return model, radio_dim


def extract_radio_features(radio_model, images_np, device="cuda", batch_size=64):
    """
    Extract RADIO summary embeddings for a batch of images.

    Args:
        radio_model: Loaded RADIO model
        images_np: numpy array (N, H, W, 3) uint8
        device: torch device
        batch_size: batch size for processing

    Returns:
        features: numpy array (N, radio_dim) float32
    """
    import torchvision.transforms as T

    # RADIO preprocessing: resize to 224x224, ImageNet normalization
    transform = T.Compose([
        T.ToPILImage(),
        T.Resize(224, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    N = images_np.shape[0]
    all_features = []

    radio_model.eval()
    dtype = next(radio_model.parameters()).dtype

    for start in range(0, N, batch_size):
        end = min(start + batch_size, N)
        batch_imgs = []
        for i in range(start, end):
            img_t = transform(images_np[i])
            batch_imgs.append(img_t)

        batch_tensor = torch.stack(batch_imgs).to(device=device, dtype=dtype)

        with torch.no_grad():
            summary, _ = radio_model(batch_tensor)

        all_features.append(summary.float().cpu())

        if (start // batch_size) % 10 == 0:
            print(f"   RADIO features: {end}/{N}")

    features = torch.cat(all_features, dim=0).numpy()
    print(f"   ✅ Extracted RADIO features: {features.shape}")
    return features


# ╔══════════════════════════════════════════════════════════════════╗
# ║  2. LANGUAGE ENCODER — NVIDIA Nemotron Nano 9B v2 (frozen)      ║
# ╚══════════════════════════════════════════════════════════════════╝

def load_nemotron_model(device="cuda"):
    """
    Load NVIDIA Nemotron Nano 9B v2 for text embedding extraction.

    Uses the hybrid Mamba-2 + Transformer architecture.
    Loaded in float16 for efficient inference on A100.

    Returns:
        model: Nemotron model
        tokenizer: Nemotron tokenizer
        hidden_dim: int, hidden dimension of the model
    """
    from transformers import AutoModelForCausalLM, AutoTokenizer

    model_name = "nvidia/NVIDIA-Nemotron-Nano-9B-v2"
    print(f"🧠 Loading {model_name} for text encoding...")
    print("   (This may take a few minutes to download ~18GB)")

    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
    )
    model.eval()

    # Detect hidden dimension from model config
    hidden_dim = model.config.hidden_size
    print(f"   ✅ Nemotron loaded — hidden dim: {hidden_dim}")

    return model, tokenizer, hidden_dim


def extract_nemotron_embedding(model, tokenizer, text, device="cuda"):
    """
    Extract a text embedding from Nemotron by mean-pooling
    the last hidden layer's representations.

    Args:
        model: Nemotron model
        tokenizer: Nemotron tokenizer
        text: instruction string

    Returns:
        embedding: numpy array (hidden_dim,) float32
    """
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
        )
        # Take the last hidden state and mean-pool over tokens
        last_hidden = outputs.hidden_states[-1]  # (1, seq_len, hidden_dim)

        if attention_mask is not None:
            mask = attention_mask.unsqueeze(-1).float()
            embedding = (last_hidden * mask).sum(dim=1) / mask.sum(dim=1)
        else:
            embedding = last_hidden.mean(dim=1)

    embedding = embedding.squeeze(0).float().cpu().numpy()
    print(f"   ✅ Nemotron text embedding: {embedding.shape}")
    return embedding


def unload_model(model):
    """Free GPU memory by deleting model and clearing cache."""
    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    import gc
    gc.collect()
    print("   🗑️  Model unloaded, GPU memory freed")


# ╔══════════════════════════════════════════════════════════════════╗
# ║  3. STATE ENCODER (trainable)                                    ║
# ╚══════════════════════════════════════════════════════════════════╝

class StateEncoder(nn.Module):
    """Encode robot proprioceptive state into a d_model-dim vector."""

    def __init__(self, state_dim: int, d_model: int = 256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.GELU(),
            nn.Linear(128, d_model),
        )
        self.ln = nn.LayerNorm(d_model)

    def forward(self, state):
        """state: (B, state_dim) -> (B, d_model)"""
        return self.ln(self.net(state))


# ╔══════════════════════════════════════════════════════════════════╗
# ║  4. CROSS-ATTENTION FUSION (trainable)                           ║
# ╚══════════════════════════════════════════════════════════════════╝

class CrossAttentionFusion(nn.Module):
    """
    Fuse vision, language, and state embeddings via cross-attention.

    Upgrade from mini-VLA's simple MLP concatenation:
    - Uses multi-head cross-attention to let modalities attend to each other
    - Followed by a feedforward network for the final fused representation
    """

    def __init__(self, d_model: int = 256, n_heads: int = 4, dropout: float = 0.1):
        super().__init__()

        # Stack vision, language, state as a 3-token sequence
        # then use self-attention to fuse them
        self.self_attn = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=n_heads,
            dropout=dropout,
            batch_first=True,
        )
        self.norm1 = nn.LayerNorm(d_model)

        # Feedforward
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 4, d_model),
            nn.Dropout(dropout),
        )
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, vis_token, txt_token, state_token):
        """
        Each input: (B, d_model)
        Returns: (B, d_model) — fused context vector
        """
        # Stack into sequence: (B, 3, d_model)
        tokens = torch.stack([vis_token, txt_token, state_token], dim=1)

        # Self-attention over the 3 modality tokens
        attn_out, _ = self.self_attn(tokens, tokens, tokens)
        tokens = self.norm1(tokens + attn_out)

        # Feedforward
        ff_out = self.ff(tokens)
        tokens = self.norm2(tokens + ff_out)

        # Mean-pool over the 3 tokens to get a single fused vector
        fused = tokens.mean(dim=1)  # (B, d_model)
        return fused


# ╔══════════════════════════════════════════════════════════════════╗
# ║  5. DIFFUSION POLICY HEAD (trainable)                            ║
# ╚══════════════════════════════════════════════════════════════════╝

@dataclass
class DiffusionConfig:
    T: int = 20                 # diffusion timesteps
    beta_start: float = 1e-4
    beta_end: float = 0.02
    action_dim: int = 4         # MetaWorld action dim
    cond_dim: int = 256         # fused context dim


def make_beta_schedule(cfg: DiffusionConfig):
    """Linear beta schedule for DDPM."""
    betas = torch.linspace(cfg.beta_start, cfg.beta_end, cfg.T)
    alphas = 1.0 - betas
    alpha_bar = torch.cumprod(alphas, dim=0)
    return betas, alphas, alpha_bar


class SinusoidalTimeEmbedding(nn.Module):
    """Sinusoidal positional encoding for diffusion timestep."""

    def __init__(self, dim: int):
        super().__init__()
        self.dim = dim

    def forward(self, t: torch.Tensor):
        """t: (B,) integer timesteps -> (B, dim)"""
        half = self.dim // 2
        freqs = torch.exp(
            torch.linspace(math.log(1.0), math.log(1000.0), half, device=t.device)
        )
        args = t.float().unsqueeze(-1) * freqs.unsqueeze(0)
        emb = torch.cat([torch.sin(args), torch.cos(args)], dim=-1)
        if self.dim % 2 == 1:
            emb = torch.cat([emb, torch.zeros_like(emb[..., :1])], dim=-1)
        return emb


class ActionDenoiseNetwork(nn.Module):
    """
    Noise prediction network: ε_θ(x_t, t, cond)
    Predicts the noise added to the action at timestep t,
    conditioned on the fused VLA context.
    """

    def __init__(self, cfg: DiffusionConfig, time_emb_dim: int = 64, hidden_dim: int = 256):
        super().__init__()
        self.time_emb = SinusoidalTimeEmbedding(time_emb_dim)

        in_dim = cfg.action_dim + time_emb_dim + cfg.cond_dim
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, cfg.action_dim),
        )

    def forward(self, x_t, t, cond):
        """
        x_t:  (B, action_dim) noisy action
        t:    (B,) integer timestep
        cond: (B, cond_dim) fused context
        """
        t_emb = self.time_emb(t)
        x = torch.cat([x_t, t_emb, cond], dim=-1)
        return self.net(x)


class DiffusionPolicyHead(nn.Module):
    """
    DDPM-based action generation head.
    Learns to denoise random Gaussian noise into robot actions,
    conditioned on the fused vision-language-state context.
    """

    def __init__(self, cfg: DiffusionConfig):
        super().__init__()
        self.cfg = cfg
        self.denoise_net = ActionDenoiseNetwork(cfg)

        betas, alphas, alpha_bar = make_beta_schedule(cfg)
        self.register_buffer("betas", betas)
        self.register_buffer("alphas", alphas)
        self.register_buffer("alpha_bar", alpha_bar)

    def q_sample(self, x0, t, noise):
        """Forward diffusion: x_t = √(ᾱ_t) * x0 + √(1-ᾱ_t) * ε"""
        ab_t = self.alpha_bar[t].unsqueeze(-1)  # (B, 1)
        return torch.sqrt(ab_t) * x0 + torch.sqrt(1.0 - ab_t) * noise

    def loss(self, actions, cond):
        """
        Training loss: MSE between predicted and true noise.

        actions: (B, action_dim) ground-truth actions
        cond:    (B, cond_dim)   fused context
        """
        B = actions.size(0)
        t = torch.randint(0, self.cfg.T, (B,), device=actions.device)
        noise = torch.randn_like(actions)
        x_t = self.q_sample(actions, t, noise)
        eps_pred = self.denoise_net(x_t, t, cond)
        return F.mse_loss(eps_pred, noise)

    @torch.no_grad()
    def sample(self, cond):
        """
        Reverse diffusion: generate actions from noise.

        cond: (B, cond_dim)
        Returns: (B, action_dim)
        """
        B = cond.size(0)
        x_t = torch.randn(B, self.cfg.action_dim, device=cond.device)

        for step in reversed(range(self.cfg.T)):
            t = torch.full((B,), step, device=cond.device, dtype=torch.long)
            eps_pred = self.denoise_net(x_t, t, cond)

            beta_t = self.betas[step]
            alpha_t = self.alphas[step]
            alpha_bar_t = self.alpha_bar[step]

            # Predict x0, then compute x_{t-1}
            x0_pred = (x_t - torch.sqrt(1 - alpha_bar_t) * eps_pred) / torch.sqrt(alpha_bar_t)

            if step > 0:
                noise = torch.randn_like(x_t)
                x_t = torch.sqrt(alpha_t) * x0_pred + torch.sqrt(beta_t) * noise
            else:
                x_t = x0_pred

        return x_t


# ╔══════════════════════════════════════════════════════════════════╗
# ║  6. NEMOTRON-VLA — Full VLA Model                               ║
# ╚══════════════════════════════════════════════════════════════════╝

class NemotronVLA(nn.Module):
    """
    Nemotron-VLA: Vision-Language-Action Model

    Architecture:
        ┌─────────────┐   ┌──────────────────┐   ┌──────────────┐
        │ NVIDIA RADIO │   │ NVIDIA Nemotron  │   │ Robot State  │
        │ (frozen ViT) │   │ Nano 9B (frozen) │   │  (raw obs)   │
        └──────┬──────┘   └────────┬─────────┘   └──────┬───────┘
               │                   │                     │
        ┌──────▼──────┐   ┌───────▼────────┐   ┌───────▼───────┐
        │ Vision Proj  │   │  Text Proj     │   │ State Encoder │
        │ Linear→LN   │   │  Linear→LN     │   │  MLP→LN       │
        └──────┬──────┘   └───────┬────────┘   └───────┬───────┘
               │                   │                     │
               └───────────┬───────┴─────────────────────┘
                           │
                    ┌──────▼──────┐
                    │ Cross-Attn  │
                    │   Fusion    │
                    └──────┬──────┘
                           │
                    ┌──────▼──────┐
                    │  Diffusion  │
                    │ Policy Head │
                    └──────┬──────┘
                           │
                      ┌────▼────┐
                      │ Actions │
                      └─────────┘

    During training:  precomputed RADIO & Nemotron embeddings → trainable layers
    During inference: RADIO runs online, Nemotron embedding is cached
    """

    def __init__(
        self,
        radio_dim: int,
        nemotron_dim: int,
        state_dim: int,
        action_dim: int = 4,
        d_model: int = 256,
        n_heads: int = 4,
        diffusion_T: int = 20,
    ):
        super().__init__()
        self.d_model = d_model

        # Projection layers for frozen encoder outputs
        self.vision_proj = nn.Sequential(
            nn.Linear(radio_dim, d_model),
            nn.LayerNorm(d_model),
        )
        self.text_proj = nn.Sequential(
            nn.Linear(nemotron_dim, d_model),
            nn.LayerNorm(d_model),
        )

        # Trainable state encoder
        self.state_encoder = StateEncoder(state_dim, d_model)

        # Cross-attention fusion
        self.fusion = CrossAttentionFusion(d_model, n_heads)

        # Diffusion action head
        diff_cfg = DiffusionConfig(
            T=diffusion_T,
            action_dim=action_dim,
            cond_dim=d_model,
        )
        self.diffusion_head = DiffusionPolicyHead(diff_cfg)

        # Save config for checkpoint
        self.config = {
            "radio_dim": radio_dim,
            "nemotron_dim": nemotron_dim,
            "state_dim": state_dim,
            "action_dim": action_dim,
            "d_model": d_model,
            "n_heads": n_heads,
            "diffusion_T": diffusion_T,
        }

    def _fuse(self, vis_emb, txt_emb, state):
        """Project and fuse all modalities."""
        vis = self.vision_proj(vis_emb)       # (B, d_model)
        txt = self.text_proj(txt_emb)         # (B, d_model)
        st = self.state_encoder(state)        # (B, d_model)
        return self.fusion(vis, txt, st)      # (B, d_model)

    def compute_loss(self, vis_emb, txt_emb, state, actions):
        """
        Training forward pass.

        Args:
            vis_emb:  (B, radio_dim)    precomputed RADIO features
            txt_emb:  (B, nemotron_dim) precomputed Nemotron features
            state:    (B, state_dim)    robot proprioceptive state
            actions:  (B, action_dim)   ground-truth expert actions

        Returns:
            loss: scalar diffusion loss
        """
        cond = self._fuse(vis_emb, txt_emb, state)
        return self.diffusion_head.loss(actions, cond)

    @torch.no_grad()
    def act(self, vis_emb, txt_emb, state):
        """
        Inference: generate action from current observation.

        Args:
            vis_emb: (1, radio_dim)    RADIO features for current image
            txt_emb: (1, nemotron_dim) cached Nemotron text embedding
            state:   (1, state_dim)    current robot state

        Returns:
            action: (1, action_dim)
        """
        self.eval()
        cond = self._fuse(vis_emb, txt_emb, state)
        return self.diffusion_head.sample(cond)


def count_parameters(model):
    """Count trainable parameters in a model."""
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable


### 2.3 `utils.py` — Utilities

In [ ]:
%%writefile utils.py
"""
Nemotron-VLA: Utilities
=======================
Dataset, training loop, evaluation, and video helpers.
"""

import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader


# ╔══════════════════════════════════════════════════════════════════╗
# ║  1. DATASET                                                      ║
# ╚══════════════════════════════════════════════════════════════════╝

class NemotronVLADataset(Dataset):
    """
    Dataset for training Nemotron-VLA with precomputed embeddings.

    Loads precomputed RADIO vision features and Nemotron text embedding,
    along with robot states and expert actions.
    """

    def __init__(self, data_path: str, embeddings_path: str):
        """
        Args:
            data_path: path to .npz with images, states, actions, instruction
            embeddings_path: path to .npz with radio_features, nemotron_embedding
        """
        # Load raw demonstration data
        raw = np.load(data_path, allow_pickle=True)
        self.states = raw["states"].astype(np.float32)    # (N, state_dim)
        self.actions = raw["actions"].astype(np.float32)   # (N, action_dim)

        # Load precomputed embeddings
        emb = np.load(embeddings_path, allow_pickle=True)
        self.vision_features = emb["radio_features"].astype(np.float32)  # (N, radio_dim)
        self.text_embedding = emb["nemotron_embedding"].astype(np.float32)  # (nemotron_dim,)

        assert len(self.states) == len(self.vision_features), \
            f"Mismatch: {len(self.states)} states vs {len(self.vision_features)} vision features"

        print(f"📦 Dataset loaded: {len(self)} samples")
        print(f"   vision features: {self.vision_features.shape}")
        print(f"   text embedding:  {self.text_embedding.shape}")
        print(f"   states:          {self.states.shape}")
        print(f"   actions:         {self.actions.shape}")

    def __len__(self):
        return len(self.states)

    def __getitem__(self, idx):
        vis = torch.from_numpy(self.vision_features[idx])
        txt = torch.from_numpy(self.text_embedding.copy())  # same for all samples
        state = torch.from_numpy(self.states[idx])
        action = torch.from_numpy(self.actions[idx])
        return vis, txt, state, action


# ╔══════════════════════════════════════════════════════════════════╗
# ║  2. TRAINING                                                     ║
# ╚══════════════════════════════════════════════════════════════════╝

def train_nemotron_vla(
    model,
    dataset,
    epochs: int = 50,
    batch_size: int = 128,
    lr: float = 3e-4,
    weight_decay: float = 1e-5,
    device: str = "cuda",
    save_path: str = "checkpoints/nemotron_vla.pt",
    log_interval: int = 5,
):
    """
    Train the Nemotron-VLA model.

    Args:
        model: NemotronVLA model
        dataset: NemotronVLADataset
        epochs: number of training epochs
        batch_size: batch size
        lr: learning rate
        weight_decay: weight decay for AdamW
        device: training device
        save_path: path to save checkpoint
        log_interval: print loss every N epochs

    Returns:
        losses: list of average loss per epoch
    """
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    model = model.to(device)
    model.train()

    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
        drop_last=True,
    )

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay,
    )

    # Cosine annealing scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=epochs,
        eta_min=lr * 0.01,
    )

    losses = []
    best_loss = float("inf")

    print(f"\n🏋️ Training Nemotron-VLA for {epochs} epochs")
    print(f"   batch_size={batch_size}, lr={lr}, samples={len(dataset)}")
    print(f"   batches/epoch={len(loader)}")
    print("-" * 60)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        n_samples = 0

        for vis, txt, state, action in loader:
            vis = vis.to(device)
            txt = txt.to(device)
            state = state.to(device)
            action = action.to(device)

            loss = model.compute_loss(vis, txt, state, action)

            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_loss += loss.item() * vis.size(0)
            n_samples += vis.size(0)

        scheduler.step()
        avg_loss = total_loss / n_samples
        losses.append(avg_loss)

        # Save best model
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save({
                "model_state_dict": model.state_dict(),
                "config": model.config,
                "epoch": epoch,
                "loss": avg_loss,
            }, save_path)

        if (epoch + 1) % log_interval == 0 or epoch == 0:
            current_lr = scheduler.get_last_lr()[0]
            print(f"   Epoch {epoch+1:3d}/{epochs} | "
                  f"loss={avg_loss:.6f} | "
                  f"best={best_loss:.6f} | "
                  f"lr={current_lr:.2e}")

    print("-" * 60)
    print(f"✅ Training complete! Best loss: {best_loss:.6f}")
    print(f"   Checkpoint saved to: {save_path}")

    return losses


# ╔══════════════════════════════════════════════════════════════════╗
# ║  3. EVALUATION                                                   ║
# ╚══════════════════════════════════════════════════════════════════╝

def evaluate_nemotron_vla(
    model,
    radio_model,
    text_embedding,
    env_name="push-v3",
    camera_name="corner2",
    seed=42,
    num_episodes=5,
    max_steps=150,
    device="cuda",
    save_video=True,
    video_dir="videos",
):
    """
    Evaluate Nemotron-VLA in the MetaWorld environment.

    Args:
        model: trained NemotronVLA
        radio_model: RADIO vision model (frozen)
        text_embedding: precomputed text embedding (nemotron_dim,)
        env_name: MetaWorld task
        num_episodes: number of eval episodes
        max_steps: max steps per episode
        device: torch device
        save_video: whether to save episode videos
        video_dir: directory for videos

    Returns:
        results: dict with rewards and success rates
    """
    import torchvision.transforms as T

    # Ensure rendering backend is configured before env import
    from env import _setup_rendering
    _setup_rendering()
    from env import MetaWorldMT1Wrapper

    # RADIO preprocessing
    radio_transform = T.Compose([
        T.ToPILImage(),
        T.Resize(224, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    radio_dtype = next(radio_model.parameters()).dtype

    # Prepare text embedding tensor (reused every step)
    txt_emb = torch.from_numpy(text_embedding).float().unsqueeze(0).to(device)

    # Create environment
    env = MetaWorldMT1Wrapper(
        env_name=env_name,
        seed=seed,
        render_mode="rgb_array",
        camera_name=camera_name,
    )

    if save_video:
        os.makedirs(video_dir, exist_ok=True)
        import imageio

    model.eval()
    radio_model.eval()

    all_rewards = []
    all_successes = []

    print(f"\n🎯 Evaluating Nemotron-VLA on {env_name}")
    print(f"   episodes={num_episodes}, max_steps={max_steps}")
    print("-" * 60)

    for ep in range(num_episodes):
        img, state, info = env.reset()
        ep_reward = 0.0
        frames = [img.copy()]
        success = False

        for step in range(max_steps):
            # Extract RADIO features for current image
            img_t = radio_transform(img).unsqueeze(0).to(device=device, dtype=radio_dtype)
            with torch.no_grad():
                summary, _ = radio_model(img_t)
                vis_emb = summary.float()  # (1, radio_dim)

            # Prepare state
            state_t = torch.from_numpy(state).float().unsqueeze(0).to(device)

            # Generate action
            with torch.no_grad():
                action_t = model.act(vis_emb, txt_emb, state_t)

            action_np = action_t.squeeze(0).cpu().numpy()

            # Step environment
            img, state, reward, done, info = env.step(action_np)
            ep_reward += reward
            frames.append(img.copy())

            if int(info.get("success", 0)) == 1:
                success = True

            if done:
                break

        all_rewards.append(ep_reward)
        all_successes.append(success)

        status = "✅ SUCCESS" if success else "❌ FAIL"
        print(f"   Episode {ep+1}/{num_episodes}: "
              f"reward={ep_reward:.2f}, steps={step+1}, {status}")

        # Save video
        if save_video:
            video_path = os.path.join(video_dir, f"{env_name}_ep{ep+1:03d}.mp4")
            with imageio.get_writer(video_path, fps=20) as writer:
                for f in frames:
                    writer.append_data(f)

    env.close()

    results = {
        "mean_reward": np.mean(all_rewards),
        "std_reward": np.std(all_rewards),
        "success_rate": np.mean(all_successes),
        "rewards": all_rewards,
        "successes": all_successes,
    }

    print("-" * 60)
    print(f"📊 Results:")
    print(f"   Mean reward:  {results['mean_reward']:.2f} ± {results['std_reward']:.2f}")
    print(f"   Success rate: {results['success_rate']*100:.1f}%")

    return results


# ╔══════════════════════════════════════════════════════════════════╗
# ║  4. CHECKPOINT LOADING                                           ║
# ╚══════════════════════════════════════════════════════════════════╝

def load_checkpoint(checkpoint_path, device="cuda"):
    """Load a trained NemotronVLA from checkpoint."""
    from models import NemotronVLA

    ckpt = torch.load(checkpoint_path, map_location=device, weights_only=False)
    cfg = ckpt["config"]

    model = NemotronVLA(
        radio_dim=cfg["radio_dim"],
        nemotron_dim=cfg["nemotron_dim"],
        state_dim=cfg["state_dim"],
        action_dim=cfg["action_dim"],
        d_model=cfg["d_model"],
        n_heads=cfg["n_heads"],
        diffusion_T=cfg["diffusion_T"],
    ).to(device)

    model.load_state_dict(ckpt["model_state_dict"])
    model.eval()

    print(f"✅ Loaded checkpoint from {checkpoint_path}")
    print(f"   Epoch: {ckpt.get('epoch', '?')}, Loss: {ckpt.get('loss', '?'):.6f}")

    return model


# ╔══════════════════════════════════════════════════════════════════╗
# ║  5. VISUALIZATION                                                ║
# ╚══════════════════════════════════════════════════════════════════╝

def plot_training_loss(losses, save_path=None):
    """Plot training loss curve."""
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(range(1, len(losses) + 1), losses, linewidth=2, color="#76b900")  # NVIDIA green
    ax.set_xlabel("Epoch", fontsize=12)
    ax.set_ylabel("Diffusion Loss", fontsize=12)
    ax.set_title("Nemotron-VLA Training Loss", fontsize=14, fontweight="bold")
    ax.grid(True, alpha=0.3)
    ax.set_yscale("log")

    if save_path:
        fig.savefig(save_path, dpi=150, bbox_inches="tight")
        print(f"   📈 Loss plot saved to {save_path}")

    plt.show()
    return fig


def display_sample_frames(data_path, n_frames=8):
    """Display sample frames from collected demonstrations."""
    import matplotlib.pyplot as plt

    data = np.load(data_path, allow_pickle=True)
    images = data["images"]
    N = len(images)

    indices = np.linspace(0, N - 1, n_frames, dtype=int)

    fig, axes = plt.subplots(1, n_frames, figsize=(2.5 * n_frames, 2.5))
    for i, idx in enumerate(indices):
        axes[i].imshow(images[idx])
        axes[i].set_title(f"t={idx}", fontsize=9)
        axes[i].axis("off")

    fig.suptitle("Expert Demonstration Samples", fontsize=13, fontweight="bold")
    plt.tight_layout()
    plt.show()
    return fig


---## 3. 🎮 Collect Expert DemonstrationsWe use MetaWorld's built-in scripted expert policy for `push-v3` to collect demonstration trajectories. Each transition includes:- **Image**: RGB observation from the `corner2` camera- **State**: 39-dim robot proprioceptive state- **Action**: 4-dim action (x, y, z, gripper)- **Instruction**: "push the object to the goal"

In [ ]:
from env import collect_demonstrations
import numpy as np

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Collect demonstrations
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
DATA_PATH = "data/push_v3_demos.npz"

if os.path.exists(DATA_PATH):
    print(f"✅ Dataset already exists at {DATA_PATH}, skipping collection")
else:
    print("🎮 Collecting expert demonstrations...")
    data = collect_demonstrations(
        env_name="push-v3",
        camera_name="corner2",
        seed=42,
        num_episodes=100,        # 100 episodes of expert play
        max_steps=150,           # up to 150 steps each
        instruction="push the object to the goal",
    )

    # Save dataset
    np.savez_compressed(
        DATA_PATH,
        images=data["images"],
        states=data["states"],
        actions=data["actions"],
        instruction=data["instruction"],
    )
    print(f"\n💾 Dataset saved to {DATA_PATH}")


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Visualize some demonstration frames
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from utils import display_sample_frames

display_sample_frames(DATA_PATH, n_frames=8)


---## 4. 🧠 Precompute NVIDIA Model EmbeddingsThis is the key memory management strategy:1. **Load NVIDIA RADIO** → extract vision features for ALL images → unload2. **Load NVIDIA Nemotron 9B** → extract text embedding → unload3. **Save** all precomputed embeddings to diskThis way, during training we only need the small trainable components in GPU memory!> ⏱️ This step takes ~5-10 minutes (downloading models + forward passes)

### 4.1 Extract RADIO Vision Features[NVIDIA RADIO (AM-RADIO)](https://huggingface.co/nvidia/RADIO) is a multi-teacher distilled vision model that combines knowledge from CLIP, DINOv2, and SAM into a single efficient backbone. We use it as a frozen feature extractor.

In [ ]:
import torch
import numpy as np
from models import load_radio_model, extract_radio_features, unload_model

EMBEDDINGS_PATH = "embeddings/precomputed.npz"
DATA_PATH = "data/push_v3_demos.npz"

if os.path.exists(EMBEDDINGS_PATH):
    print(f"✅ Embeddings already exist at {EMBEDDINGS_PATH}, skipping")
    emb = np.load(EMBEDDINGS_PATH, allow_pickle=True)
    radio_dim = emb["radio_features"].shape[1]
    nemotron_dim = emb["nemotron_embedding"].shape[0]
    print(f"   RADIO dim:    {radio_dim}")
    print(f"   Nemotron dim: {nemotron_dim}")
else:
    # Load demonstration images
    data = np.load(DATA_PATH, allow_pickle=True)
    images = data["images"]  # (N, H, W, 3)
    instruction = str(data["instruction"])
    print(f"Loaded {len(images)} images and instruction: '{instruction}'")

    # ── Step 1: RADIO Vision Features ──
    print("\n" + "="*60)
    print("STEP 1: NVIDIA RADIO Vision Features")
    print("="*60)

    radio_model, radio_dim = load_radio_model(device="cuda", dtype=torch.float16)
    radio_features = extract_radio_features(
        radio_model, images, device="cuda", batch_size=64
    )
    unload_model(radio_model)

    print(f"\n📸 RADIO features shape: {radio_features.shape}")
    print(f"   GPU memory after unload: {torch.cuda.memory_allocated()/1e9:.2f} GB")


### 4.2 Extract Nemotron Text Embedding[NVIDIA Nemotron Nano 9B v2](https://huggingface.co/nvidia/NVIDIA-Nemotron-Nano-9B-v2) uses a hybrid **Mamba-2 + Transformer** architecture for high-throughput language understanding. We extract a text embedding by mean-pooling the last hidden layer.Since we use a single instruction for all episodes, we only need to compute this embedding **once**.

In [ ]:
if not os.path.exists(EMBEDDINGS_PATH):
    # ── Step 2: Nemotron Text Embedding ──
    print("\n" + "="*60)
    print("STEP 2: NVIDIA Nemotron Nano 9B Text Embedding")
    print("="*60)

    from models import load_nemotron_model, extract_nemotron_embedding

    nemotron_model, tokenizer, nemotron_dim = load_nemotron_model(device="cuda")
    nemotron_embedding = extract_nemotron_embedding(
        nemotron_model, tokenizer, instruction, device="cuda"
    )
    unload_model(nemotron_model)
    del tokenizer

    print(f"\n🧠 Nemotron embedding shape: {nemotron_embedding.shape}")
    print(f"   GPU memory after unload: {torch.cuda.memory_allocated()/1e9:.2f} GB")

    # ── Save all embeddings ──
    np.savez_compressed(
        EMBEDDINGS_PATH,
        radio_features=radio_features,
        nemotron_embedding=nemotron_embedding,
        radio_dim=radio_dim,
        nemotron_dim=nemotron_dim,
    )
    print(f"\n💾 All embeddings saved to {EMBEDDINGS_PATH}")
    print(f"   RADIO features:    {radio_features.shape}")
    print(f"   Nemotron embedding: {nemotron_embedding.shape}")


---## 5. 🏋️ Train Nemotron-VLANow we train only the lightweight components:- **Vision projection**: Linear layer mapping RADIO features → d_model- **Text projection**: Linear layer mapping Nemotron embedding → d_model- **State encoder**: 2-layer MLP for robot state- **Cross-attention fusion**: Multi-head attention over modality tokens- **Diffusion policy head**: DDPM-based action generationTotal trainable parameters: **~0.8M** — trains fast even without the A100 being fully utilized!

In [ ]:
import torch
from models import NemotronVLA, count_parameters
from utils import NemotronVLADataset, train_nemotron_vla

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Load dataset with precomputed embeddings
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
dataset = NemotronVLADataset(
    data_path="data/push_v3_demos.npz",
    embeddings_path="embeddings/precomputed.npz",
)

# Load embedding dimensions
emb = np.load("embeddings/precomputed.npz", allow_pickle=True)
radio_dim = int(emb["radio_dim"])
nemotron_dim = int(emb["nemotron_dim"])

# Infer state and action dims from data
raw = np.load("data/push_v3_demos.npz", allow_pickle=True)
state_dim = raw["states"].shape[1]
action_dim = raw["actions"].shape[1]

print(f"\n🔧 Model Configuration:")
print(f"   RADIO dim:    {radio_dim}")
print(f"   Nemotron dim: {nemotron_dim}")
print(f"   State dim:    {state_dim}")
print(f"   Action dim:   {action_dim}")


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Create Nemotron-VLA model
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
model = NemotronVLA(
    radio_dim=radio_dim,
    nemotron_dim=nemotron_dim,
    state_dim=state_dim,
    action_dim=action_dim,
    d_model=256,             # internal representation dim
    n_heads=4,               # cross-attention heads
    diffusion_T=20,          # diffusion denoising steps
)

total_params, trainable_params = count_parameters(model)
print(f"\n📊 Nemotron-VLA Model:")
print(f"   Total parameters:     {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Memory footprint:     ~{trainable_params * 4 / 1e6:.1f} MB (float32)")


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Train!
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
losses = train_nemotron_vla(
    model=model,
    dataset=dataset,
    epochs=80,               # training epochs
    batch_size=128,          # batch size (can increase on A100)
    lr=3e-4,                 # learning rate
    weight_decay=1e-5,       # AdamW weight decay
    device="cuda",
    save_path="checkpoints/nemotron_vla.pt",
    log_interval=5,          # print every 5 epochs
)


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Plot training loss
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from utils import plot_training_loss

plot_training_loss(losses, save_path="checkpoints/loss_curve.png")


---## 6. 🎯 Evaluate Nemotron-VLANow we test the trained model in the MetaWorld `push-v3` environment. During inference:- **RADIO** runs online to process each new camera frame- **Nemotron** embedding is cached (loaded from disk)- The **fusion + diffusion head** generates actions in real-time

In [ ]:
import torch
import numpy as np
from models import load_radio_model
from utils import load_checkpoint, evaluate_nemotron_vla

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Load trained model and RADIO for online inference
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
device = "cuda"

# Load trained Nemotron-VLA
vla_model = load_checkpoint("checkpoints/nemotron_vla.pt", device=device)

# Load RADIO for online vision processing
radio_model, _ = load_radio_model(device=device, dtype=torch.float16)

# Load cached text embedding
emb = np.load("embeddings/precomputed.npz", allow_pickle=True)
text_embedding = emb["nemotron_embedding"]

print(f"\n✅ All models loaded for inference")
print(f"   GPU memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Run evaluation
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
results = evaluate_nemotron_vla(
    model=vla_model,
    radio_model=radio_model,
    text_embedding=text_embedding,
    env_name="push-v3",
    camera_name="corner2",
    seed=42,
    num_episodes=10,
    max_steps=150,
    device=device,
    save_video=True,
    video_dir="videos",
)


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Display a recorded video
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from IPython.display import HTML
from base64 import b64encode
import glob

video_files = sorted(glob.glob("videos/*.mp4"))
if video_files:
    # Display the first video
    video_path = video_files[0]
    with open(video_path, "rb") as f:
        video_data = b64encode(f.read()).decode("ascii")

    display(HTML(f'''
    <h3>🎬 Nemotron-VLA: {os.path.basename(video_path)}</h3>
    <video width="480" controls autoplay loop>
        <source src="data:video/mp4;base64,{video_data}" type="video/mp4">
    </video>
    '''))
else:
    print("No videos found in videos/ directory")


---## 7. 📊 Summary & Next Steps### What We Built**Nemotron-VLA** — a Vision-Language-Action model for robot manipulation using NVIDIA's foundation models:| Component | Model | Source ||-----------|-------|--------|| Vision Encoder | NVIDIA RADIO | [HuggingFace](https://huggingface.co/nvidia/RADIO) || Language Encoder | NVIDIA Nemotron Nano 9B v2 | [HuggingFace](https://huggingface.co/nvidia/NVIDIA-Nemotron-Nano-9B-v2) || Fusion Module | Cross-Attention (custom) | Trained from scratch || Action Head | DDPM Diffusion Policy (custom) | Trained from scratch |### Key Design Decisions1. **Precomputed embeddings**: We extract features from RADIO and Nemotron offline, enabling training on A100 40GB2. **Cross-attention fusion**: Upgraded from mini-VLA's MLP concatenation to multi-head self-attention over modality tokens3. **Deeper diffusion head**: 3-layer noise prediction network with LayerNorm for better action quality4. **Memory efficiency**: Only ~0.8M trainable parameters, entire training fits in <4GB GPU memory### Possible Extensions- 🔄 **Multi-task**: Train on multiple MetaWorld tasks (MT10/MT50)- 🗣️ **Arbitrary instructions**: Load Nemotron at inference for free-form text- 📈 **Action chunking**: Predict sequences of future actions- 🔬 **LoRA fine-tuning**: Fine-tune RADIO or Nemotron with LoRA for domain adaptation- 🤖 **Real robot**: Deploy on physical hardware with camera input---*Built with ❤️ using NVIDIA Nemotron & RADIO foundation models*